In [435]:
wikipedia_page = 'List_of_postal_codes_of_Canada:_M'
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

%config IPCompleter.greedy = True
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

table_from_top = 1

trace = False
wikipedia_url = 'https://en.wikipedia.org/wiki/{}'.format(wikipedia_page)
page = requests.get(wikipedia_url)
soup = BeautifulSoup(page.content, 'lxml')
tables = soup.find_all('table', {'class': 'wikitable'})
table = tables[table_from_top - 1]

feature_names = []

header_row = table.find('tr')
for header in header_row.find_all('th'):
    feature_name = ' '.join(header.find_all(text=True))
    feature_name.replace('\n', '')
    feature_names.append(feature_name)
    
def has_coords(tag):
    if tag.has_attr('class'):
        if tag['class'][0] == 'latitude' or tag['class'][0] == 'longitude':
            return True
    return False

def get_coords(child):
    coords = []
    for coord in child.find_all(has_coords):
        coords.append(coord.string)
    if coords:
        if trace:
            return 'C = {}'.format(' '.join(coords))
        else:
            return ' '.join(coords)
    else:
        return ''

samples = []
sample_rows = table.find_all('tr')[1:]
for sample_row in sample_rows:
    features = []
    for feature_col in sample_row.find_all('td'):
        feature_value = ''
        text = feature_col.string
        if text:
            if trace:
                features.append('T = {}'.format(text))
            else:
                features.append(text)
            continue
        
        for child in feature_col.children:
            if child.name == 'span':
                if child.has_attr('class'):
                    if child['class'] == 'display:none':
                        continue
                if child.find_all(has_coords):
                    feature_value = get_coords(child)
                    if feature_value:
                        break
                    else:
                        continue
            if child.name == 'sup':
                continue
            if child.name == 'a':
                if child.string[0] == '[':
                    continue            
            if child.name == 'a':
                if trace:
                    feature_value = 'A = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            if child.name == 'font':
                if trace:
                    feature_value = 'F = {}'.format(child.string)
                else:
                    feature_value = child.string
                break
            try:
                content = child.contents
            except AttributeError:
                if child.isspace():
                    continue
                if trace:
                    feature_value = 'E = {}'.format(child)
                else:
                    feature_value = child
                break
        features.append(feature_value)
    samples.append(dict(zip(feature_names, features)))

'Postcode'

'Borough'

'Neighbourhood'

In [456]:
df = pd.DataFrame(samples)
df.columns = df.columns.str.replace('\n','')
df.columns = df.columns.str.replace('Postcode','PostalCode')
df = df.replace({'\n' : ''})
#df1 = df.T.reindex(['PostalCode','Borough','Neighbourhood']).T
df = df[df.Borough != 'Not assigned' ]
df = df.dropna()
df = df.reset_index(drop=True)
#df.head()

In [437]:
df = df.replace({'Not assigned': ''}, regex=True)
#df = df.groupby('PostalCode').agg({'Borough':'sum','Neighbourhood':','.join})
df = df.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.astype(str))))

df = df.replace({'\n' : ','})
df['Neighbourhood'] = df['Neighbourhood'].replace(',', df['Borough'])
df.loc[df['Borough'] == "Queen's Park"]

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [438]:
df.shape

(103, 3)

In [457]:
df = df.replace({'\n,' : ''})
df.head()

,Borough,Neighbourhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


In [458]:
import pandas as pd
dt = pd.read_csv('http://cocl.us/Geospatial_data')
dt.columns = dt.columns.str.replace('Postal Code','PostalCode')
dt.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [462]:
dfl = pd.merge(df, dt[['PostalCode', 'Latitude', 'Longitude']], on='PostalCode', how='left')
#dfl
dfl

,Borough,Neighbourhood,PostalCode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
3,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
4,North York,Lawrence Heights,M6A,43.718518,-79.464763
5,North York,Lawrence Manor,M6A,43.718518,-79.464763
6,Queen's Park,Not assigned,M7A,43.662301,-79.389494
7,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
8,Scarborough,Rouge,M1B,43.806686,-79.194353
9,Scarborough,Malvern,M1B,43.806686,-79.194353


In [ ]:

!conda install -c conda-forge folium=0.5.0 --yes
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

print('Folium installed')
print('Libraries imported.')

In [469]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
   
# tranforming json file into a pandas dataframe library
import json # library to handle JSON files
from pandas.io.json import json_normalize


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [470]:
CLIENT_ID = 'JHN451BKG4FJOAJR0RZ5ONIGTYNOMGP1RDNVROCZDVA3S2HN' # your Foursquare ID
CLIENT_SECRET = 'U4FLNGFTDCQO5ZBPIBM1P5OHRX0YR3YQTANKU12XRHYZXTEF' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JHN451BKG4FJOAJR0RZ5ONIGTYNOMGP1RDNVROCZDVA3S2HN
CLIENT_SECRET:U4FLNGFTDCQO5ZBPIBM1P5OHRX0YR3YQTANKU12XRHYZXTEF


In [471]:
df = df[df['Borough'].str.contains("Toronto")]
df = df.dropna()
df = df.reset_index(drop=True)
dfl = pd.merge(df, dt[['PostalCode', 'Latitude', 'Longitude']], on='PostalCode', how='left')
#dfl
dfl.head()
dfl.shape

,Borough,Neighbourhood,PostalCode,Latitude,Longitude
0,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
1,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
2,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937
3,Downtown Toronto,Garden District,M5B,43.657162,-79.378937
4,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418


(74, 5)

## 1. Explore Dataset

In [478]:
neighborhoods = dfl
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 4 boroughs and 74 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto City.

In [480]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


# create map of Toronto using latitude and longitude values

In [483]:

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfl['Latitude'], dfl['Longitude'], dfl['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto 

In [484]:
CLIENT_ID = 'JHN451BKG4FJOAJR0RZ5ONIGTYNOMGP1RDNVROCZDVA3S2HN' # your Foursquare ID
CLIENT_SECRET = 'U4FLNGFTDCQO5ZBPIBM1P5OHRX0YR3YQTANKU12XRHYZXTEF' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JHN451BKG4FJOAJR0RZ5ONIGTYNOMGP1RDNVROCZDVA3S2HN
CLIENT_SECRET:U4FLNGFTDCQO5ZBPIBM1P5OHRX0YR3YQTANKU12XRHYZXTEF


#### Let's explore the first neighborhood in our dataframe.

In [486]:
manhattan_data = dfl
manhattan_data.loc[0, 'Neighbourhood']
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

'Harbourfront'

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Harbourfront within a radius of 1000 meters.

In [488]:
radius = 1000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()
results

'https://api.foursquare.com/v2/venues/explore?&client_id=JHN451BKG4FJOAJR0RZ5ONIGTYNOMGP1RDNVROCZDVA3S2HN&client_secret=U4FLNGFTDCQO5ZBPIBM1P5OHRX0YR3YQTANKU12XRHYZXTEF&v=20180604&ll=43.6542599,-79.3606359&radius=1000&limit=30'

{'meta': {'code': 200, 'requestId': '5c6fc5eedb04f505a1ca4c77'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 115,
  'suggestedBounds': {'ne': {'lat': 43.66325990900001,
    'lng': -79.3482199002972},
   'sw': {'lat': 43.64525989099999, 'lng': -79.37305189970282}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label':

In [489]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [492]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues.head()
nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,The Distillery Historic District,Historic Site,43.650244,-79.359323
5,Distillery Sunday Market,Farmers Market,43.650075,-79.361832
6,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
7,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
8,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127
9,Body Blitz Spa East,Spa,43.654735,-79.359874


In [493]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

In [500]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [501]:
Toronto_venues = getNearbyVenues(names=manhattan_data['Neighbourhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District

St. James Town
The Beaches
Berczy Park
Central Bay Street

Christie

Adelaide
King
Richmond

Dovercourt Village
Dufferin

Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West

Riverdale
Design Exchange
Toronto Dominion Centre
Brockton

Exhibition Place
Parkdale Village
The Beaches West

India Bazaar
Commerce Court
Victoria Hotel

Studio District

Lawrence Park
Roselawn

Davisville North

Forest Hill North
Forest Hill West

High Park
The Junction South

North Toronto West

The Annex
North Midtown

Yorkville
Parkdale
Roncesvalles
Davisville

Harbord

University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East

Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE

Rathnelly
South Hill
Summerhill West

CN Tower
Bathurst Quay

Island airport

Harbourfront West

King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade

Cabbagetown
St. James Town
First Canadi

In [503]:
print(Toronto_venues.shape)
Toronto_venues.head()
Toronto_venues.groupby('Neighborhood').count()


(1619, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,30,30,30,30,30,30
Bathurst Quay,14,14,14,14,14,14
Berczy Park,30,30,30,30,30,30
Brockton,19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
CN Tower,14,14,14,14,14,14
Cabbagetown,30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Chinatown,30,30,30,30,30,30


In [504]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 190 uniques categories.


## 3. Analyze Each Neighborhood

In [506]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [507]:
Toronto_onehot.shape

(1619, 190)

In [508]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [509]:
Toronto_grouped.shape

(73, 190)

#### Let's print each neighborhood along with the top 5 most common venues

In [510]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0           Steakhouse  0.10
1                Hotel  0.07
2                 Café  0.07
3     Asian Restaurant  0.07
4  American Restaurant  0.07


----Bathurst Quay
----
              venue  freq
0  Airport Terminal  0.14
1    Airport Lounge  0.14
2   Airport Service  0.14
3   Harbor / Marina  0.07
4  Sculpture Garden  0.07


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.10
1  Seafood Restaurant  0.07
2              Bakery  0.07
3                Café  0.07
4      Farmers Market  0.07


----Brockton
----
                venue  freq
0      Breakfast Spot  0.11
1                Café  0.11
2         Coffee Shop  0.11
3  Italian Restaurant  0.05
4       Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern
----
           venue  freq
0    Yoga Studio  0.06
1  Auto Workshop  0.06
2    Pizza Place  0.06
3           Park  0.06
4     Restaurant  0.06


----CN Tower----
              venue  freq
0  Air

In [511]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [512]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Steakhouse,Hotel,American Restaurant,Asian Restaurant,Café,Sushi Restaurant,Seafood Restaurant,Bar,Smoke Shop,Lounge
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boutique,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boat or Ferry
2,Berczy Park,Cocktail Bar,Café,Seafood Restaurant,Farmers Market,Bakery,Park,Concert Hall,Comfort Food Restaurant,Liquor Store,Museum
3,Brockton,Coffee Shop,Café,Breakfast Spot,Furniture / Home Store,Burrito Place,Caribbean Restaurant,Bar,Convenience Store,Italian Restaurant,Pet Store
4,Business Reply Mail Processing Centre 969 East...,Yoga Studio,Butcher,Skate Park,Smoke Shop,Restaurant,Spa,Brewery,Farmers Market,Fast Food Restaurant,Auto Workshop


## 4. Cluster Neighborhoods

In [513]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0], dtype=int32)

In [526]:
dfl.columns = dfl.columns.str.replace('Neighbourhood','Neighborhood')
dfl.head()
neighborhoods_venues_sorted.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636
1,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636
2,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937
3,Downtown Toronto,Garden District,M5B,43.657162,-79.378937
4,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418


,ClusterLabel,ClusterLabels1,ClusterLabels,Cluster Labels1,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,0,0,0,0,Adelaide,Steakhouse,Hotel,American Restaurant,Asian Restaurant,Café,Sushi Restaurant,Seafood Restaurant,Bar,Smoke Shop,Lounge
1,1,1,1,1,1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boutique,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boat or Ferry
2,0,0,0,0,0,Berczy Park,Cocktail Bar,Café,Seafood Restaurant,Farmers Market,Bakery,Park,Concert Hall,Comfort Food Restaurant,Liquor Store,Museum
3,0,0,0,0,0,Brockton,Coffee Shop,Café,Breakfast Spot,Furniture / Home Store,Burrito Place,Caribbean Restaurant,Bar,Convenience Store,Italian Restaurant,Pet Store
4,0,0,0,0,0,Business Reply Mail Processing Centre 969 East...,Yoga Studio,Butcher,Skate Park,Smoke Shop,Restaurant,Spa,Brewery,Farmers Market,Fast Food Restaurant,Auto Workshop


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [527]:
Toronto_data = dfl
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabel3', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Borough,Neighborhood,PostalCode,Latitude,Longitude,ClusterLabel3,ClusterLabel,ClusterLabels1,ClusterLabels,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,M5A,43.654260,-79.360636,0,0,0,0,0,0,Coffee Shop,Park,Bakery,Café,Pub,Mexican Restaurant,Breakfast Spot,Italian Restaurant,Performing Arts Venue,Chocolate Shop
1,Downtown Toronto,Regent Park,M5A,43.654260,-79.360636,0,0,0,0,0,0,Coffee Shop,Park,Bakery,Café,Pub,Mexican Restaurant,Breakfast Spot,Italian Restaurant,Performing Arts Venue,Chocolate Shop
2,Downtown Toronto,Ryerson,M5B,43.657162,-79.378937,0,0,0,0,0,0,Café,Coffee Shop,Clothing Store,Japanese Restaurant,Concert Hall,Comic Shop,Mexican Restaurant,Diner,Movie Theater,Shopping Mall
3,Downtown Toronto,Garden District,M5B,43.657162,-79.378937,0,0,0,0,0,0,Café,Coffee Shop,Clothing Store,Japanese Restaurant,Concert Hall,Comic Shop,Mexican Restaurant,Diner,Movie Theater,Shopping Mall
4,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,0,0,0,0,0,0,Italian Restaurant,Restaurant,Coffee Shop,Gastropub,Japanese Restaurant,Café,Hotel,Thai Restaurant,Diner,Deli / Bodega


# Create Map

In [530]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['ClusterLabel3']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [537]:
#Toronto_merged.drop(['Cluster Labels'], axis=1, inplace=True)
cls0 = Toronto_merged.loc[Toronto_merged['ClusterLabel3'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cls1 = Toronto_merged.loc[Toronto_merged['ClusterLabel3'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cls2 = Toronto_merged.loc[Toronto_merged['ClusterLabel3'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cls3 = Toronto_merged.loc[Toronto_merged['ClusterLabel3'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cls4 = Toronto_merged.loc[Toronto_merged['ClusterLabel3'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cls5 = Toronto_merged.loc[Toronto_merged['ClusterLabel3'] == 5, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

### Restaurant Cluster

In [540]:
cls0 = cls0.dropna()
cls0 = cls0.reset_index(drop=True)
#df.head()
cls0

,Neighborhood,ClusterLabel3,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,0,Coffee Shop,Park,Bakery,Café,Pub,Mexican Restaurant,Breakfast Spot,Italian Restaurant,Performing Arts Venue,Chocolate Shop
1,Regent Park,0,Coffee Shop,Park,Bakery,Café,Pub,Mexican Restaurant,Breakfast Spot,Italian Restaurant,Performing Arts Venue,Chocolate Shop
2,Ryerson,0,Café,Coffee Shop,Clothing Store,Japanese Restaurant,Concert Hall,Comic Shop,Mexican Restaurant,Diner,Movie Theater,Shopping Mall
3,Garden District,0,Café,Coffee Shop,Clothing Store,Japanese Restaurant,Concert Hall,Comic Shop,Mexican Restaurant,Diner,Movie Theater,Shopping Mall
4,St. James Town,0,Italian Restaurant,Restaurant,Coffee Shop,Gastropub,Japanese Restaurant,Café,Hotel,Thai Restaurant,Diner,Deli / Bodega
5,The Beaches,0,Coffee Shop,Health Food Store,Pub,Wine Bar,Creperie,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
6,Berczy Park,0,Cocktail Bar,Café,Seafood Restaurant,Farmers Market,Bakery,Park,Concert Hall,Comfort Food Restaurant,Liquor Store,Museum
7,Central Bay Street,0,Coffee Shop,Italian Restaurant,Café,Spa,Park,Steakhouse,Modern European Restaurant,Chinese Restaurant,Pizza Place,Bubble Tea Shop
8,Christie,0,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Diner,Nightclub,Convenience Store,Italian Restaurant,Baby Store
9,Adelaide,0,Steakhouse,Hotel,American Restaurant,Asian Restaurant,Café,Sushi Restaurant,Seafood Restaurant,Bar,Smoke Shop,Lounge


### Airport Cluster

In [547]:
cls1 = cls1.dropna()
cls1 = cls1.reset_index(drop=True)
#df.head()
cls1

,Neighborhood,ClusterLabel3,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,CN Tower,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boat or Ferry
1,Bathurst Quay,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boat or Ferry
2,Island airport,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boat or Ferry
3,Harbourfront West,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boat or Ferry
4,King and Spadina,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boat or Ferry
5,Railway Lands,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boat or Ferry
6,South Niagara,1,Airport Lounge,Airport Service,Airport Terminal,Boutique,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boat or Ferry


### Garden Restaurant Cluster

In [542]:
cls2 = cls2.dropna()
cls2 = cls2.reset_index(drop=True)
#df.head()
cls2

,Neighborhood,ClusterLabel3,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Roselawn,2,Garden,Wine Bar,Creperie,Dumpling Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega


### Park Cluster

In [543]:
cls3 = cls3.dropna()
cls3 = cls3.reset_index(drop=True)
#df.head()
cls3

,Neighborhood,ClusterLabel3,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Forest Hill North,3,Park,Trail,Jewelry Store,Sushi Restaurant,Wine Bar,Creperie,Dog Run,Discount Store,Diner,Dim Sum Restaurant
1,Forest Hill West,3,Park,Trail,Jewelry Store,Sushi Restaurant,Wine Bar,Creperie,Dog Run,Discount Store,Diner,Dim Sum Restaurant
2,Rosedale,3,Park,Playground,Trail,Wine Bar,Coworking Space,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


In [549]:
### Play Ground Cluster

In [544]:
cls4 = cls4.dropna()
cls4 = cls4.reset_index(drop=True)
#df.head()
cls4

,Neighborhood,ClusterLabel3,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Moore Park,4,Restaurant,Playground,Coworking Space,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega,Dance Studio
1,Summerhill East,4,Restaurant,Playground,Coworking Space,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega,Dance Studio
